## Test Case Strategy code Paralized

## 1. Imports

In [1]:
import os
import sys
import math
import glob
import os.path

from math import log10
from argparse import ArgumentParser
from os.path import join, basename, dirname
import healpy as hp
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib

import astropy.units as u
import matplotlib.pyplot as plt

from astropy.cosmology import z_at_value
from scipy.stats import uniform
from scipy.stats import norm
from scipy.interpolate import interp1d
from astropy.cosmology import WMAP9 as cosmo

### 1.a Custom import

In [2]:
from kn_strategy_sims_parallel import KNCalc, calc_mag_fractions

## 2 If name == 'main' part

In [3]:
teff_kind = 'moony'  # 'notmoony'

options = {
    'distance': 150,
    'distance_err': 50,
    'time_delay': 60.0,
    'fraction': 90,
    'magplot_file': 'kn_mag_plot_clecio_loglan5.png',
    'expplot_file': 'kn_exp_plot_clecio_test_loglan5.png',
    'report_file': 'kn_report_clecio_test',
    'filter': None
}

distances = [50, 75, 150, 200, 250, 300]
loglan = -1.0
kntype = 'blue'  # 'blue'
kn_weights = True
w_type = "gaussian_narrow"  # 'gaussian'#

event_path = './'
out_dir = './'
sufix = '_'+teff_kind+'_'+kntype+'_'

event_list = glob.glob(join(event_path, '*.fits.gz'))
map_mode = 1  # 1 for map 0 for dist
area_covered_deep = [0.8, 0.7, 0.5, 0.3, 0.7, 0.5, 0.3, 0.5, 0.4, 0.3]
multi_exposure_run = True
multi_exposure = multi_exposure_run

if multi_exposure_run:
    area_covered = [0.9, 0.9, 0.9, 0.9, 0.8, 0.8, 0.8, 0.7, 0.7, 0.7]
else:
    area_covered = [0.9, 0.85, 0.8, 0.75, 0.7]

if map_mode:
    second_loop = area_covered
    second_loop_legend = "Region Coverage"
    areas_cov_deep = area_covered_deep
    distances = [250.0 for i in range(0, len(area_covered))]
else:
    second_loop = distances
    eventmap = ""
    event_list = [""]
    second_loop_legend = "Distance (MPC)"
    m_exp = False
    m_exp_run = False

if not multi_exposure:
    area_covered_deep = [0.0 for i in range(0, len(second_loop))]

plot_dist_name = 'detection_distance_loglan5.png'
plot_td_name = 'detection_time_delay_loglan5.png'
time_delays = [12.0, 24.0, 36.0, 48.0, 60.0, 72.0, 84.0, 96.0]

filters_comb = ['gg', 'gr', 'gi', 'gz', 'rg', 'rr', 'ri',
                'rz', 'ig', 'ir', 'ii', 'iz', 'zg', 'zr', 'zi', 'zz']

if not multi_exposure:
    exposure_times_calc = [60.0, 90.0, 120.0,
                           200.0, 300.0, 600.0, 1200.0, 2400.0, 3600.0]
else:
    exposure_times_calc = [60.0, 90.0, 120.0, 200.0, 300.0,
                           300.0, 600.0, 600.0, 1200.0, 1200, 2400.0, 3600.0, 300.0]

exposure_times_calc_deep = [90.0, 120.0, 200.0, 300.0, 600.0,
                            1200.0, 1200.0, 2400.0, 2400.0, 3600.0, 3600.0, 5400.0, 5400.0]

# shape (4,13)
maglims_exp = np.zeros((len(['g', 'r', 'i', 'z']),
                        len(exposure_times_calc)))
# shape (4,13)
maglims_exp_l = np.zeros((len(['g', 'r', 'i', 'z']),
                          len(exposure_times_calc)))
# shape (4, 13, 10, 8)
fractions_all = np.zeros((len(['g', 'r', 'i', 'z']),
                          len(exposure_times_calc),
                          len(second_loop),
                          len(time_delays)))
# shape (4, 13, 10, 8)
fractions_all_later = np.zeros((len(['g', 'r', 'i', 'z']),
                                len(exposure_times_calc),
                                len(second_loop),
                                len(time_delays)),)

day_delays = np.array(time_delays)/24.0
hours_per_night = 8.0
time_per_night_sec = hours_per_night*60*60  # 21600
mjd_correction = True
second_loop

[0.9, 0.9, 0.9, 0.9, 0.8, 0.8, 0.8, 0.7, 0.7, 0.7]

In [12]:
def parallize_event(time_delay_ac, options: dict, e, map_mode=1):
    
        time_delay, second_loop_element = time_delay_ac
        options['distance'] = 250.0
        options['time_delay'] = time_delay
        if map_mode == 1:
            _map_file = e
        else:
            _map_file = ""  # e
        m_exp = multi_exposure_run
        
        kn_calc = KNCalc(
            e,
            float(options['distance']),
            float(options['distance_err']),
            float(options['time_delay']),
            float(loglan),
            kn_weight_type=w_type,
            plot=False,
            use_map=True,
            area_covered=second_loop_element,
            reduce_mapresolution=True,
            set_mjd_correction=mjd_correction,
            m_exp_kncalc=m_exp,
            deep_coverage=second_loop_element
        )

#         if (kn_calc.sw_mexp == True) and (m_exp_run == True):
#             warning_area_deep.append(area_covered_deep[i])

#         if kn_calc.Flag == 0:
#             print(' I will pass')
#             continue
#         m_exp = kn_calc.mult_exp

#         if map_mode:
#             area_deg_arr[i] = kn_calc.area_deg
#             if m_exp == True:
#                 area_deg_arr_deep[i] = kn_calc.area_deg_deep

#         else:
#             area_deg_arr[i] = area_deg_fix

        if m_exp == False:
            percentile_dict = calc_mag_fractions(
                kn_calc.template_df_full,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type
            )

            percentile_dict_later = calc_mag_fractions(
                kn_calc.template_df_later,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type
            )

        else:
            percentile_dict, percentile_dict_deep = calc_mag_fractions(
                kn_calc.template_df_full,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type,
                m_exp=m_exp
            )

            percentile_dict_later, percentile_dict_later_deep = calc_mag_fractions(
                kn_calc.template_df_full,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type,
                m_exp=m_exp
            )
# area_deg_arr = area_deg_arr.reshape(8,10)
# area_deg_arr_deep = area_deg_arr_deep.reshape(8,10)

In [26]:
import multiprocessing as mpl

for e in event_list:
    if map_mode == 1:

        print(" ============================== \n ==============================\n")
        print(f"starting the run for file {e}")
        print(" ============================== \n ==============================\n")
        plot_name_ = basename(e)
        plot_name_ = plot_name_.rstrip(".fits.gz")
        plot_name_ = plot_name_.rstrip(".fits")
        plot_name_ = out_dir+plot_name_+sufix
    else:
        plot_name_ = out_dir+"GW_sim"+sufix

    if os.path.isfile(plot_name_+"_allconfig.csv"):
        print('This event event strategy already exists. Skipping')
        continue

    warning_area_deep = []
    
    time_delay_second_loop_comb = [
        (time_delay, area_covered) \
        for time_delay in time_delays \
        for area_covered in second_loop
    ]
    
    area_deg_arr = np.zeros(len(time_delay_second_loop_comb))
    area_deg_arr_deep = np.zeros(len(time_delay_second_loop_comb))
    for i, time_delay_ac in enumerate(time_delay_second_loop_comb):

        p = mpl.Process(parallize_event, args=(time_delay_ac, options, e))
        p.start()
        p.join()


starting the run for file ./bayestar.fits.gz



AssertionError: group argument must be None for now

In [4]:
for e in event_list:
    if map_mode == 1:

        print(" ============================== \n ==============================\n")
        print(f"starting the run for file {e}")
        print(" ============================== \n ==============================\n")
        plot_name_ = basename(e)
        plot_name_ = plot_name_.rstrip(".fits.gz")
        plot_name_ = plot_name_.rstrip(".fits")
        plot_name_ = out_dir+plot_name_+sufix
    else:
        plot_name_ = out_dir+"GW_sim"+sufix

    if os.path.isfile(plot_name_+"_allconfig.csv"):
        print('This event event strategy already exists. Skipping')
        continue

    warning_area_deep = []
    
    time_delay_second_loop_comb = [
        (time_delay, area_covered) \
        for time_delay in time_delays \
        for area_covered in second_loop
    ]
    
    area_deg_arr = np.zeros(len(time_delay_second_loop_comb))
    area_deg_arr_deep = np.zeros(len(time_delay_second_loop_comb))
    
    for i, time_delay_ac in enumerate(time_delay_second_loop_comb):
        print(f'loop number: {i+1}')
        time_delay, second_loop_element = time_delay_ac
        options['distance'] = 250.0
        options['time_delay'] = time_delay
        if map_mode == 1:
            _map_file = e
        else:
            _map_file = ""  # e
        m_exp = multi_exposure_run
        
        kn_calc = KNCalc(
            e,
            float(options['distance']),
            float(options['distance_err']),
            float(options['time_delay']),
            float(loglan),
            kn_weight_type=w_type,
            plot=False,
            use_map=True,
            area_covered=second_loop_element,
            reduce_mapresolution=True,
            set_mjd_correction=mjd_correction,
            m_exp_kncalc=m_exp,
            deep_coverage=second_loop_element
        )

        if (kn_calc.sw_mexp == True) and (m_exp_run == True):
            warning_area_deep.append(area_covered_deep[i])

        if kn_calc.Flag == 0:
            print(' I will pass')
            continue
        m_exp = kn_calc.mult_exp

        if map_mode:
            area_deg_arr[i] = kn_calc.area_deg
            if m_exp == True:
                area_deg_arr_deep[i] = kn_calc.area_deg_deep

        else:
            area_deg_arr[i] = area_deg_fix

        if m_exp == False:
            percentile_dict = calc_mag_fractions(
                kn_calc.template_df_full,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type
            )

            percentile_dict_later = calc_mag_fractions(
                kn_calc.template_df_later,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type
            )

        else:
            percentile_dict, percentile_dict_deep = calc_mag_fractions(
                kn_calc.template_df_full,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type,
                m_exp=m_exp
            )

            percentile_dict_later, percentile_dict_later_deep = calc_mag_fractions(
                kn_calc.template_df_full,
                use_knmodel_weights=kn_weights,
                kn_type=kntype,
                kn_weight_type=w_type,
                m_exp=m_exp
            )
area_deg_arr = area_deg_arr.reshape(8,10)
area_deg_arr_deep = area_deg_arr_deep.reshape(8,10)


starting the run for file ./bayestar.fits.gz

loop number: 1
Using preprocessed weights for  ./weights/bayestarac0.9ad0.9.npy
===== Using KN model priors gaussian_narrow , considering gw170817 with blue component
loading KN model weights ...
===== Using KN model priors gaussian_narrow , considering gw170817 with blue component
loading KN model weights ...
loop number: 2
Using preprocessed weights for  ./weights/bayestarac0.9ad0.9.npy
===== Using KN model priors gaussian_narrow , considering gw170817 with blue component
loading KN model weights ...
===== Using KN model priors gaussian_narrow , considering gw170817 with blue component
loading KN model weights ...
loop number: 3
Using preprocessed weights for  ./weights/bayestarac0.9ad0.9.npy
===== Using KN model priors gaussian_narrow , considering gw170817 with blue component
loading KN model weights ...
===== Using KN model priors gaussian_narrow , considering gw170817 with blue component
loading KN model weights ...
loop number: 4
Us

In [5]:
kncalc.

array([[3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458,
        3.35717458, 3.35717458, 3.35717458, 3.35717458, 3.35717458],
       [3.35717458, 3.35717458, 3.3571745

In [8]:
kn_calc.area_deg_deep

23.50022206591267